# Quickstart: SerDes link exploration
This notebook demonstrates the modular simulation pipeline provided by the `serdes` package.
Adjust the configuration values below to explore jitter, ISI, and equalization behavior.

In [ ]:
import sys, pathlib

repo_root = pathlib.Path.cwd()
while repo_root != repo_root.parent:
    if (repo_root / 'serdes').exists():
        break
    repo_root = repo_root.parent
else:
    raise RuntimeError("Could not locate repository root containing 'serdes'.")

repo_root_str = str(repo_root)
if repo_root_str not in sys.path:
    sys.path.insert(0, repo_root_str)


In [ ]:
import matplotlib.pyplot as plt

from serdes import (
    CdrConfig,
    ChannelConfig,
    EyeConfig,
    JitterConfig,
    RxEqualizerConfig,
    SerDesPipeline,
    SignalConfig,
    TxEqualizerConfig,
)
from serdes.plotting import plot_bathtub, plot_eye, plot_waveform

In [ ]:
signal = SignalConfig(symbol_rate=56e9, samples_per_symbol=32, modulation='pam4')
tx = TxEqualizerConfig(taps=(0.12, 1.0, -0.14))
channel = ChannelConfig(alpha=0.18, post_cursor=10)
rx = RxEqualizerConfig(ctle_zero_hz=12e9, ctle_pole_hz=38e9, dfe_taps=(0.05, -0.03, 0.01))
jitter = JitterConfig(rj_sigma_ui=0.008, sj_amplitude_ui=0.02, sj_frequency=7e6)
cdr = CdrConfig(loop_bandwidth_hz=18e6, damping=0.8, gain=0.05)
eye = EyeConfig(bins=180)

pipeline = SerDesPipeline(signal=signal, tx=tx, channel=channel, rx=rx, jitter=jitter, cdr=cdr, eye=eye)
result = pipeline.run(num_symbols=2048)

result.equalized_symbols[:8]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
plot_waveform(result.waveform_rx.time, result.waveform_rx.samples, ax=axes[0])
plot_eye(result.eye, ax=axes[1])
plot_bathtub(result.bathtub, ax=axes[2])
plt.tight_layout()
plt.show()